In [1]:
import numpy as np
import physbo
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def load_data():
    A = np.asarray(np.loadtxt("data/s5-210.csv", skiprows=1, delimiter=","))
    X = A[:, 0:3]
    t = -A[:, 3]
    return X, t

In [3]:
# Load the data
# X is the N x d dimensional matrix. Each row of X denotes the d-dimensional feature vector of search candidate.
# t is the N-dimensional vector that represents the corresponding negative energy of search candidates.
# ( It is of course unknown in practice. )
X, t = load_data()

# Normalize the mean and standard deviation along the each column of X to 0 and 1, respectively
X = physbo.misc.centering(X)

In [4]:
# Declare the class for calling the simulator.
# In this tutorial, we simply refer to the value of t.
# If you want to apply physbo to other problems, you have to customize this class.
class simulator:
    def __init__(self):
        _, self.t = load_data()

    def __call__(self, action):
        return self.t[action]

In [5]:
# Design of policy

# Declaring the policy by
policy = physbo.search.discrete.policy(test_X=X)
# test_X is the set of candidates which is represented by numpy.array.
# Each row vector represents the feature vector of the corresponding candidate

# set the seed parameter
policy.set_seed(0)

In [6]:
# If you want to perform the initial random search before starting the Bayesian optimization,
# the random sampling is performed by

res = policy.random_search(
    max_num_probes=2, num_search_each_probe=10, simulator=simulator()
)
# Input:
# max_num_probes: number of random search
# num_search_each_probe: number of probes
# simulator:  simulator which is defined above
# output: physbo.search.discreate.results.history (class)


# multiple probe Bayesian search
# The single query version of COMBO is performed by
res = policy.bayes_search(
    max_num_probes=8,
    num_search_each_probe=10,
    simulator=simulator(),
    score="EI",
    interval=2,
    num_rand_basis=0,
)

# Input
# max_num_probes: number of searching by Bayesian optimization
# num_search_each_probe: number of probes
# simulator: the class of simulator which is defined above
# score: the type of aquision funciton. TS, EI and PI are available
# interval: the timing for learning the hyper parameter.
#               In this case, the hyper parameter is learned at each 20 steps
#               If you set the negative value to interval, the hyper parameter learning is not performed
#               If you set zero to interval, the hyper parameter learning is performed only at the first step
# num_rand_basis: the number of basis function. If you choose 0,  ordinary Gaussian process runs

0001-th multiple probe search (random) 

current best f(x) = -0.980054 (best action = 4547) 
list of simulation results
f(x)=-1.070602 (action = 15673)
f(x)=-1.009056 (action = 9559)
f(x)=-1.195844 (action = 16927)
f(x)=-0.980054 (action = 4547)
f(x)=-0.992820 (action = 2553)
f(x)=-1.146676 (action = 13144)
f(x)=-1.006255 (action = 10827)
f(x)=-0.999862 (action = 1995)
f(x)=-1.055445 (action = 10763)
f(x)=-1.100970 (action = 16450)


0002-th multiple probe search (random) 

current best f(x) = -0.980054 (best action = 4547) 
list of simulation results
f(x)=-1.208666 (action = 13085)
f(x)=-1.069404 (action = 15133)
f(x)=-1.031642 (action = 1706)
f(x)=-1.016702 (action = 2464)
f(x)=-1.172569 (action = 17812)
f(x)=-1.082219 (action = 16533)
f(x)=-1.025272 (action = 1336)
f(x)=-1.031761 (action = 10076)
f(x)=-0.984972 (action = 8876)
f(x)=-1.107730 (action = 15577)


Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0 -th epoch marginal likelihood

In [7]:
# The result of searching is summarized in the class physbo.search.discrete.results.history()
# res.fx: observed negative energy at each step
# res.chosen_actions: history of choosed actions
# fbest, best_action= res.export_sequence_best_fx(): current best fx and current best action
#                                                                                                   that has been observed until each probe
# res.num_runs: number of probes
# res.total_num_search: total number of actions
print("f(x)=")
print(res.fx[0 : res.total_num_search])

best_fx, best_action = res.export_sequence_best_fx()

print("current best at each probe")
print(best_fx)
print("current best action at each probe=")
print(best_action)
print("history of chosen actions=")
print(res.chosen_actions[0 : res.total_num_search])

f(x)=
[-1.07060214 -1.00905602 -1.1958441  -0.98005416 -0.99281999 -1.14667577
 -1.00625502 -0.99986158 -1.05544467 -1.10097025 -1.20866565 -1.06940391
 -1.03164227 -1.01670171 -1.17256876 -1.08221892 -1.02527215 -1.03176082
 -0.98497219 -1.1077304  -1.08031283 -0.96123802 -1.08748007 -1.06988832
 -1.01522317 -1.04468073 -1.04504063 -1.01930058 -1.01869507 -1.01162164
 -1.02510471 -0.97718778 -0.97176629 -0.98458443 -1.0177939  -1.14570866
 -1.28362776 -1.01511657 -0.9781168  -0.96369703 -0.96468774 -0.97829476
 -0.98097921 -0.9744533  -1.04621299 -0.99167549 -1.05196509 -0.96376526
 -0.96952617 -1.04916921 -0.96599372 -0.97478363 -1.0130527  -1.05603682
 -0.99770846 -0.97555914 -0.99828355 -0.99681497 -0.96852892 -1.12481102
 -0.97487289 -0.99222727 -0.98350446 -1.01301176 -0.9595784  -1.03037382
 -0.9852775  -2.49732771 -1.07149506 -0.95913748 -0.97795732 -0.97233883
 -1.07941309 -0.97864414 -0.9659761  -0.99977033 -2.83283038 -1.05232556
 -1.03199421 -2.4971452  -1.04669399 -1.14092